In [2]:
# check version of gluonts
# import gluonts
# gluonts.__version__

In [1]:
# !pip install gluonts=="0.4.1"
# change to 0.4.0 if any problems occur
import gluonts
gluonts.__version__

'0.4.1'

In [2]:
import mxnet
mxnet.__version__

'1.4.1'

In [3]:
# Imports
import pprint
from functools import partial

import pandas as pd

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deepstate import DeepStateEstimator

from gluonts.trainer import Trainer

import mxnet as mx
import numpy as np

INFO:root:Using CPU
INFO:root:Using CPU


In [8]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 400
num_batches_per_epoch = 400
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=400, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=400, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='3M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='24000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=8>, train=<gluonts.dataset.common.File

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 400/400 [00:40<00:00,  9.78it/s, avg_epoch_loss=7.44]
INFO:root:Epoch[0] Elapsed time 40.894 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.435294
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 400/400 [00:41<00:00,  9.71it/s, avg_epoch_loss=7.16]
INFO:root:Epoch[1] Elapsed time 41.216 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.159251
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 400/400 [00:50<00:00,  7.90it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[2] Elapsed time 50.623 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.939147
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 400/400 [00:42<00:00,  9.50it/s, avg_epoch_loss=6.9] 
INFO:root:Epoch[3] Elapsed time 42.085 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.899338
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 400/400 [00:45<00:00,  8.89it/s, a

INFO:root:Epoch[73] Learning rate is 0.001
100%|██████████| 400/400 [00:44<00:00,  9.08it/s, avg_epoch_loss=6.15]
INFO:root:Epoch[73] Elapsed time 44.085 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.146993
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 400/400 [00:44<00:00,  9.06it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[74] Elapsed time 44.142 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.243522
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 400/400 [00:42<00:00,  9.39it/s, avg_epoch_loss=6.19]
INFO:root:Epoch[75] Elapsed time 42.621 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.189233
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 400/400 [00:43<00:00,  9.17it/s, avg_epoch_loss=6.25]
INFO:root:Epoch[76] Elapsed time 43.601 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.251016
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 400/400 [00:46<00:00,  8.54it/s, avg_epoch_loss=6.13

INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=6.082886
INFO:root:Epoch[145] Learning rate is 6.25e-05
100%|██████████| 400/400 [00:27<00:00, 14.53it/s, avg_epoch_loss=6.16]
INFO:root:Epoch[145] Elapsed time 27.533 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=6.162062
INFO:root:Epoch[146] Learning rate is 6.25e-05
100%|██████████| 400/400 [00:26<00:00, 14.88it/s, avg_epoch_loss=6.06]
INFO:root:Epoch[146] Elapsed time 26.905 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=6.059016
INFO:root:Loading parameters from best epoch (116)
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 400/400 [00:29<00:00, 13.38it/s, avg_epoch_loss=6.2] 
INFO:root:Epoch[147] Elapsed time 29.928 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.198476
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 400/400 [00:26<00:00, 15.12it/s, avg_epoch_loss=6.09]
INFO:root:Epoch[148] Elapsed time 26.466 seconds
INFO:root:Epoch[148] Evaluation metric 'epo

...

INFO:root:Epoch[146] Learning rate is 0.001
100%|██████████| 400/400 [00:31<00:00, 12.63it/s, avg_epoch_loss=5.78]
INFO:root:Epoch[146] Elapsed time 31.685 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=5.777936
INFO:root:Epoch[147] Learning rate is 0.001
100%|██████████| 400/400 [00:31<00:00, 12.84it/s, avg_epoch_loss=5.58]
INFO:root:Epoch[147] Elapsed time 31.152 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.580756
INFO:root:Epoch[148] Learning rate is 0.001
100%|██████████| 400/400 [00:35<00:00, 11.27it/s, avg_epoch_loss=5.92]
INFO:root:Epoch[148] Elapsed time 35.505 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=5.923767
INFO:root:Epoch[149] Learning rate is 0.001
100%|██████████| 400/400 [00:31<00:00, 12.60it/s, avg_epoch_loss=5.59]
INFO:root:Epoch[149] Elapsed time 31.754 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=5.591182
INFO:root:Epoch[150] Learning rate is 0.001
100%|██████████| 400/400 [00:30<00:00, 12.91it/s, avg_ep

100%|██████████| 400/400 [01:14<00:00,  5.40it/s, avg_epoch_loss=5.56]
INFO:root:Epoch[218] Elapsed time 74.023 seconds
INFO:root:Epoch[218] Evaluation metric 'epoch_loss'=5.559968
INFO:root:Epoch[219] Learning rate is 0.0005
100%|██████████| 400/400 [01:20<00:00,  4.97it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[219] Elapsed time 80.478 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=5.383350
INFO:root:Epoch[220] Learning rate is 0.0005
100%|██████████| 400/400 [01:22<00:00,  4.87it/s, avg_epoch_loss=5.71]
INFO:root:Epoch[220] Elapsed time 82.193 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=5.710388
INFO:root:Epoch[221] Learning rate is 0.0005
100%|██████████| 400/400 [01:34<00:00,  4.22it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[221] Elapsed time 94.706 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=5.365207
INFO:root:Epoch[222] Learning rate is 0.0005
100%|██████████| 400/400 [00:42<00:00,  9.49it/s, avg_epoch_loss=5.73]
INFO:root:Epoch[222] Elap

INFO:root:Epoch[289] Evaluation metric 'epoch_loss'=5.233929
INFO:root:Epoch[290] Learning rate is 0.000125
100%|██████████| 400/400 [00:23<00:00, 16.78it/s, avg_epoch_loss=5.55]
INFO:root:Epoch[290] Elapsed time 23.839 seconds
INFO:root:Epoch[290] Evaluation metric 'epoch_loss'=5.552509
INFO:root:Epoch[291] Learning rate is 0.000125
100%|██████████| 400/400 [00:38<00:00, 10.34it/s, avg_epoch_loss=5.22]
INFO:root:Epoch[291] Elapsed time 38.698 seconds
INFO:root:Epoch[291] Evaluation metric 'epoch_loss'=5.223530
INFO:root:Epoch[292] Learning rate is 0.000125
100%|██████████| 400/400 [00:36<00:00, 11.05it/s, avg_epoch_loss=5.59]
INFO:root:Epoch[292] Elapsed time 36.211 seconds
INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=5.592802
INFO:root:Epoch[293] Learning rate is 0.000125
100%|██████████| 400/400 [00:26<00:00, 15.18it/s, avg_epoch_loss=5.22]
INFO:root:Epoch[293] Elapsed time 26.362 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=5.219381
INFO:root:Epoch[294] Learni

INFO:root:Epoch[361] Elapsed time 21.203 seconds
INFO:root:Epoch[361] Evaluation metric 'epoch_loss'=5.222427
INFO:root:Epoch[362] Learning rate is 5e-05
100%|██████████| 400/400 [00:24<00:00, 16.42it/s, avg_epoch_loss=5.55]
INFO:root:Epoch[362] Elapsed time 24.366 seconds
INFO:root:Epoch[362] Evaluation metric 'epoch_loss'=5.552649
INFO:root:Epoch[363] Learning rate is 5e-05
100%|██████████| 400/400 [00:30<00:00, 13.18it/s, avg_epoch_loss=5.14]
INFO:root:Epoch[363] Elapsed time 30.352 seconds
INFO:root:Epoch[363] Evaluation metric 'epoch_loss'=5.138605
INFO:root:Epoch[364] Learning rate is 5e-05
100%|██████████| 400/400 [00:27<00:00, 14.78it/s, avg_epoch_loss=5.53]
INFO:root:Epoch[364] Elapsed time 27.078 seconds
INFO:root:Epoch[364] Evaluation metric 'epoch_loss'=5.532617
INFO:root:Epoch[365] Learning rate is 5e-05
100%|██████████| 400/400 [00:35<00:00, 11.28it/s, avg_epoch_loss=5.29]
INFO:root:Epoch[365] Elapsed time 35.464 seconds
INFO:root:Epoch[365] Evaluation metric 'epoch_loss'

{'Coverage[0.1]': 0.17602173913043295,
 'Coverage[0.2]': 0.2546304347826083,
 'Coverage[0.3]': 0.32131884057971144,
 'Coverage[0.4]': 0.38607971014492826,
 'Coverage[0.5]': 0.4497753623188407,
 'Coverage[0.6]': 0.507746376811595,
 'Coverage[0.7]': 0.5771521739130449,
 'Coverage[0.8]': 0.6526376811594249,
 'Coverage[0.9]': 0.7462318840579759,
 'MAE_Coverage': 0.08137198067632702,
 'MASE': 3.1986303682054515,
 'MSE': 3468365.993967756,
 'MSIS': 55.87422971689149,
 'ND': 0.13961258340552568,
 'NRMSE': 0.29859139680550306,
 'QuantileLoss[0.1]': 54610029.029015355,
 'QuantileLoss[0.2]': 81706728.11801147,
 'QuantileLoss[0.3]': 100577959.90260848,
 'QuantileLoss[0.4]': 113120816.24083252,
 'QuantileLoss[0.5]': 120167996.15660095,
 'QuantileLoss[0.6]': 122011332.72144775,
 'QuantileLoss[0.7]': 117619691.31997375,
 'QuantileLoss[0.8]': 105495138.82471925,
 'QuantileLoss[0.9]': 81801718.59312744,
 'RMSE': 1862.3549591760848,
 'abs_error': 120167996.1289978,
 'abs_target_mean': 6237.135359895143

In [11]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]

,dataset,estimator,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9]
0,m4_quarterly,DeepAREstimator,1.143213,0.099424,12.033866,0.092270,0.050172
1,m4_yearly,DeepAREstimator,3.198630,0.143194,55.874230,0.139613,0.095038


In [9]:
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb
print("Done")

[NbConvertApp] Converting notebook GluonTS09-m4_data_deepar_deepstate.ipynb to html
[NbConvertApp] Writing 503942 bytes to ./html_outputs\GluonTS09-m4_data_deepar_deepstate.html


Done


In [10]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 500
num_batches_per_epoch = 400
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=500, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=400, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='3M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='24000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=8>, train=<gluonts.dataset.common.File

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/400 [00:09<?, ?it/s, avg_epoch_loss=7.91]


KeyboardInterrupt: 

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb
print("Done")

In [ ]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 1000
num_batches_per_epoch = 400
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb
print("Done")

In [ ]:
print("Done!")